# PikaBot Is Back With a Vengeance 
> Indirect Syscalls and Layers of Crypto

- toc: true 
- badges: true
- categories: [pikabot,debugging,string decryption]

## Overview

According to Zscaler, the core Pikabot module includes the ability to execute arbitrary commands and inject payloads that are provided by a command-and-control server. 

A new updated version of the bot has been observed in the wild. This new version includes significant anti-analysis features that may have been added as response to the initial analysis and reporting on earlier version of the bot.

### Sample
`39d6f7865949ae7bb846f56bff4f62a96d7277d2872fec68c09e1227e6db9206` [UnpacMe](https://www.unpac.me/results/671ae130-6070-48fc-82f2-afc305fdbf3d?hash=39d6f7865949ae7bb846f56bff4f62a96d7277d2872fec68c09e1227e6db9206#/)

### References 

- [PikaBot - Tiny loader that seems very familiar](https://research.openanalysis.net/pikabot/yara/config/loader/2023/02/26/pikabot.html)
- [Technical Analysis of Pikabot (older version)](https://www.zscaler.com/blogs/security-research/technical-analysis-pikabot)


## Analysis

The core module is injected into a legitimate `SearchProtocolHost.exe` process which is launched by the loader. The loader uses indirect syscalls to hide it's injection but once observed these are just simple process injection calls like `NTWriteVirtualMemory`. 

To further hide the core module the binary has been built to mimic a legitimate`SearchProtocolHost.exe` binary including some of the plain text strings in the binary. 

```
LegalCopyright © Microsoft Corporation. All rights reserved.
InternalName SearchProtocolHost.exe
FileVersion 7.0.19041.1151 (WinBuild.160101.0800)
CompanyName Microsoft Corporation
ProductName Windows® Search
ProductVersion 7.0.19041.1151
FileDescription Microsoft Windows Search Protocol Host
OriginalFilename SearchProtocolHost.exe
charsetID 1200
Translation 0x0409 0x04b0
LangID 0x0409
```

## Strings

- The strings are in-line stack strings that are encrypted with RC4 (in-line). 
- The decrypted strings are base64 encoded 
- Once base64 decoded they are then decrypted again using AES CBC
- The key and iv are hard coded... they look like base64 strings but they are used as plaintext keys/iv
- The key and iv strings are longer than required and only the first 16/32 bytes are used, possibly as a way to further hide them


### Example

key (only first 32 bytes used) `l9SpFBoXEyglbY0ginoTUBd=pP=y6rVcQG8tP/zV4iqr06yZKEb+VCg1yQJ5jUNE`

iv (only first 16 bytes used) `FdbAwsDj0FJcgkLPb1J/mqGU7T6e98p9CMnoB`

`osEN0n6vv65dChRx0nCwRA__` -> `osEN0n6vv65dChRx0nCwRA==` -> decrypted to `kernel32`



### Automated String Decryption (Debugger)
We can automate some of the string decryption by adding a logging breakpoint on the string decrypt functions. Syntax for logging in x64dbg follows. The `@@@@` is just used as a unique string to find these in the logs and can be filtered out.


#### IDA Labels 
The following script can be used to add string labels to IDA (rebased to 0x00F90000)

In [ ]:

import idaapi, idc, idautils

strings = [{ "a1":0xF9AB9F, "a2":0xFA6A29,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA6A29,  "value":'''GetUserDefaultLangID''' },
{ "a1":0xF9AB9F, "a2":0xF93662,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF93662,  "value":'''CreateMutexW''' },
{ "a1":0xF9AB9F, "a2":0xF93672,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF93672,  "value":'''GetLastError''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA66F5, "a2":0x15D4E6AA,  "value":'''C:\\''' },
{ "a1":0xF9AB9F, "a2":0xFA6710,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA6710,  "value":'''GetVolumeInformationW''' },
{ "a1":0xF9AB9F, "a2":0xF953C5,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF953C5,  "value":'''GetComputerNameW''' },
{ "a1":0xF9AB9F, "a2":0xF950B5,  "value":'''Advapi32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF950B5,  "value":'''GetUserNameW''' },
{ "a1":0xF9AB9F, "a2":0xF94F62,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF94F62,  "value":'''GetProductInfo''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF94F6C, "a2":0x0,  "value":'''\%d''' },
{ "a1":0xF9AB9F, "a2":0xF94F80,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF94F80,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA6787, "a2":0x15D4E6AA,  "value":'''\%s\\\%s|\%s''' },
{ "a1":0xF9AB9F, "a2":0xFA67A0,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA67A0,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA685D, "a2":0x15D4E6AA,  "value":'''\%07lX\%09lX\%lu''' },
{ "a1":0xF9AB9F, "a2":0xFA6878,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA6878,  "value":'''wsprintfW''' },
{ "a1":0xF9371D, "a2":0x3157537A,  "value":'''\%s''' },
{ "a1":0xF9AB9F, "a2":0xF9373B,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9373B,  "value":'''wsprintfA''' },
{ "a1":0xF93762, "a2":0x3157537A,  "value":'''&''' },
{ "a1":0xF9AB9F, "a2":0xF94B7E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF94B7E,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF94F62,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF94F62,  "value":'''GetProductInfo''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF94F6C, "a2":0x31366B33,  "value":'''\%d''' },
{ "a1":0xF9AB9F, "a2":0xF94F80,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF94F80,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF950B5,  "value":'''Advapi32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF950B5,  "value":'''GetUserNameW''' },
{ "a1":0xF9AB9F, "a2":0xF953C5,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF953C5,  "value":'''GetComputerNameW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF9AB9F, "a2":0xF95E45,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF95E45,  "value":'''EnumDisplayDevicesW''' },
{ "a1":0xF9AB9F, "a2":0xF96015,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF96015,  "value":'''GlobalMemoryStatusEx''' },
{ "a1":0xF9AB9F, "a2":0xF969E6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF969E6,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF96E81,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF96E81,  "value":'''GetCurrentProcess''' },
{ "a1":0xF9AB9F, "a2":0xF96E92,  "value":'''Advapi32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF96E92,  "value":'''OpenProcessToken''' },
{ "a1":0xF9AB9F, "a2":0xF96EB5,  "value":'''Advapi32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF96EB5,  "value":'''GetTokenInformation''' },
{ "a1":0xF9AB9F, "a2":0xF96FBE,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF96FBE,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF965CF,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF965CF,  "value":'''GetDesktopWindow''' },
{ "a1":0xF9AB9F, "a2":0xF967CC,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF967CC,  "value":'''GetWindowRect''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF967D9, "a2":0x51187093,  "value":'''\%dx\%d''' },
{ "a1":0xF9AB9F, "a2":0xF967F3,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF967F3,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF956CF,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF956CF,  "value":'''GetComputerNameExW''' },
{ "a1":0xF9AB9F, "a2":0xF95A19,  "value":'''NetApi32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF95A19,  "value":'''DsGetDcNameW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF95B33, "a2":0x3858696A,  "value":'''unknown''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA8D5A, "a2":0x30487233,  "value":'''{"mdPNC6f8": "\%s", "NUn3h77h": "\%s", "W381C": "Win \%d.\%d \%d", "SJ3sWSeKQ": \%s, "YlSwktC": "\%s", "7smbRjIVo": "\%s", "AU3KNl": "\%s", "hgcPNL3D": \%d, "RMe4xUeMl": "\%s", "K92GpjSw": \%d, "04FjIN": "\%s", "yVqTFN": "\%s", "TrKEIz": "\%s", "gh5jjQAh": "\%s", "EoLYU": "\%s", "gBmbRM40z": \%d, "NUqeH": \%d}''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA8D67, "a2":0x30487233,  "value":'''GG9TU@T@f0adda360d2b4ccda11468e026526576''' },
{ "a1":0xF9AB9F, "a2":0xFA8DEC,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA8DEC,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xFA8E91, "a2":0x684C4B6F,  "value":'''&tfDgx=''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA8E9B, "a2":0x684C4B6F,  "value":'''whoami.exe /all''' },
{ "a1":0xF9AB9F, "a2":0xFB2DAF,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2DAF,  "value":'''CreatePipe''' },
{ "a1":0xF9AB9F, "a2":0xFB2E6D,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2E6D,  "value":'''CreateProcessW''' },
{ "a1":0xF9AB9F, "a2":0xFB34B2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB34B2,  "value":'''WaitForSingleObject''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB34B2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB34B2,  "value":'''WaitForSingleObject''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB38E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38E2,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB38F4,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38F4,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3906,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3906,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3918,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3918,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' },
{ "a1":0xFA8E91, "a2":0x68366265,  "value":'''&M1LWU=''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA8E9B, "a2":0x68366265,  "value":'''ipconfig.exe /all''' },
{ "a1":0xF9AB9F, "a2":0xFB2DAF,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2DAF,  "value":'''CreatePipe''' },
{ "a1":0xF9AB9F, "a2":0xFB2E6D,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2E6D,  "value":'''CreateProcessW''' },
{ "a1":0xF9AB9F, "a2":0xFB34B2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB34B2,  "value":'''WaitForSingleObject''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB38E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38E2,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB38F4,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38F4,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3906,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3906,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3918,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3918,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' },
{ "a1":0xFA8E91, "a2":0x63777074,  "value":'''&VC76f=''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA8E9B, "a2":0x63777074,  "value":'''netstat.exe -aon''' },
{ "a1":0xF9AB9F, "a2":0xFB2DAF,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2DAF,  "value":'''CreatePipe''' },
{ "a1":0xF9AB9F, "a2":0xFB2E6D,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2E6D,  "value":'''CreateProcessW''' },
{ "a1":0xF9AB9F, "a2":0xFB34B2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB34B2,  "value":'''WaitForSingleObject''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB383E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB383E,  "value":'''ReadFile''' },
{ "a1":0xF9AB9F, "a2":0xFB3807,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3807,  "value":'''PeekNamedPipe''' },
{ "a1":0xF9AB9F, "a2":0xFB38E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38E2,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB38F4,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB38F4,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3906,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3906,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFB3918,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB3918,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' },
{ "a1":0xFA8F71, "a2":0x4855364C,  "value":'''&SBSlO=''' },
{ "a1":0xF9AB9F, "a2":0xFB1FC8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB1FC8,  "value":'''CreateToolhelp32Snapshot''' },
{ "a1":0xF9AB9F, "a2":0xFB2000,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2000,  "value":'''Process32FirstW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFB212E, "a2":0x590016,  "value":'''[''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFB213E, "a2":0x590016,  "value":'''"\%s:\%d:\%d:\%d:\%d:\%d:\%d"''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFB214C, "a2":0x590016,  "value":''', "\%s:\%d:\%d:\%d:\%d:\%d:\%d"''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFB215A, "a2":0x590016,  "value":''']''' },
{ "a1":0xF9AB9F, "a2":0xFB2172,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2172,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB11C8,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB11C8,  "value":'''IsWow64Process''' },
{ "a1":0xF9AB9F, "a2":0xF9CC36,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9CC36,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xFB2417,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB2417,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB24D0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24D0,  "value":'''Process32NextW''' },
{ "a1":0xF9AB9F, "a2":0xFB24FC,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB24FC,  "value":'''wsprintfW''' },
{ "a1":0xF9AB9F, "a2":0xFB258E,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFB258E,  "value":'''CloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xF9AB9F, "a2":0xF988E0,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF988E0,  "value":'''GetTickCount''' },
{ "a1":0xFA929A, "a2":0x730069,  "value":'''AV89JS''' },
{ "a1":0xFA92AB, "a2":0x730069,  "value":'''&''' },
{ "a1":0xF9E523, "a2":0xF0A26600,  "value":'''\%s&\%s''' },
{ "a1":0xF9E530, "a2":0xF0A26600,  "value":'''UndoubtableEthnologically=antitwilightFluidextract&birefractingUndeceitfulness=huehuetl&CryptocurrencyMarcionitic=h1RxpQSFbBJ''' },
{ "a1":0xF9E53D, "a2":0xF0A26600,  "value":'''UdvGU=''' },
{ "a1":0xF9AB9F, "a2":0xF9E558,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9E558,  "value":'''wsprintfA''' },
{ "a1":0xF9AB9F, "a2":0xFA13EF,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA13EF,  "value":'''InternetOpenW''' },
{ "a1":0xFA145C, "a2":0x41734430,  "value":'''&''' },
{ "a1":0xF9AB9F, "a2":0xFA162D,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA162D,  "value":'''InternetConnectW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA18D1, "a2":0x41734430,  "value":'''POST''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA18DB, "a2":0x41734430,  "value":'''TrichinopolyUncontriving/uiDV6mKfgGakdg?unshelledSplitnut=vEzlHkL''' },
{ "a1":0xF9AB9F, "a2":0xFA18F9,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA18F9,  "value":'''HttpOpenRequestW''' },
{ "a1":0xF9AB9F, "a2":0xFA1C79,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1C79,  "value":'''InternetQueryOptionW''' },
{ "a1":0xF9AB9F, "a2":0xFA1DCA,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1DCA,  "value":'''InternetSetOptionW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xF9AB9F, "a2":0xFA1DEB,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1DEB,  "value":'''lstrlenW''' },
{ "a1":0xF9AB9F, "a2":0xFA1E00,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1E00,  "value":'''lstrlenA''' },
{ "a1":0xF9AB9F, "a2":0xFA1E19,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1E19,  "value":'''HttpSendRequestW''' },
{ "a1":0xF9AB9F, "a2":0xFA22D8,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22D8,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFA22EB,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22EB,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFA22FB,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22FB,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' },
{ "a1":0xF9E523, "a2":0xF0A26600,  "value":'''\%s&\%s''' },
{ "a1":0xF9E530, "a2":0xF0A26600,  "value":'''UndoubtableEthnologically=antitwilightFluidextract&birefractingUndeceitfulness=huehuetl&CryptocurrencyMarcionitic=h1RxpQSFbBJ''' },
{ "a1":0xF9E53D, "a2":0xF0A26600,  "value":'''UdvGU=''' },
{ "a1":0xF9AB9F, "a2":0xF9E558,  "value":'''User32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9E558,  "value":'''wsprintfA''' },
{ "a1":0xF9AB9F, "a2":0xFA13EF,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA13EF,  "value":'''InternetOpenW''' },
{ "a1":0xFA145C, "a2":0x41734430,  "value":'''&''' },
{ "a1":0xF9AB9F, "a2":0xFA162D,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA162D,  "value":'''InternetConnectW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA18D1, "a2":0x41734430,  "value":'''POST''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA18DB, "a2":0x41734430,  "value":'''TrichinopolyUncontriving/uiDV6mKfgGakdg?unshelledSplitnut=vEzlHkL''' },
{ "a1":0xF9AB9F, "a2":0xFA18F9,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA18F9,  "value":'''HttpOpenRequestW''' },
{ "a1":0xF9AB9F, "a2":0xFA1C79,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1C79,  "value":'''InternetQueryOptionW''' },
{ "a1":0xF9AB9F, "a2":0xFA1DCA,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1DCA,  "value":'''InternetSetOptionW''' },
{ "a1":0xF9AB9F, "a2":0xF9B8E2,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9B8E2,  "value":'''MultiByteToWideChar''' },
{ "a1":0xFA1DD7, "a2":0x41734430,  "value":'''Content-Type: application/x-www-form-urlencoded \nAccept: */* \nAccept-Language: en-US,en;q=0.5 \nAccept-Encoding: gzip, deflate \nUser-Agent: Mozilla/5.0 (compatible; MSIE 5.0; Windows NT 5.01; Trident/5.1)'''}, 
{ "a1":0xF9AB9F, "a2":0xFA1DEB,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1DEB,  "value":'''lstrlenW''' },
{ "a1":0xF9AB9F, "a2":0xFA1E00,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1E00,  "value":'''lstrlenA''' },
{ "a1":0xF9AB9F, "a2":0xFA1E19,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA1E19,  "value":'''HttpSendRequestW''' },
{ "a1":0xF9AB9F, "a2":0xFA22D8,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22D8,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFA22EB,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22EB,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xFA22FB,  "value":'''Wininet.dll''' },
{ "a1":0xF9ABD2, "a2":0xFA22FB,  "value":'''InternetCloseHandle''' },
{ "a1":0xF9AB9F, "a2":0xF9C5B6,  "value":'''Kernel32.dll''' },
{ "a1":0xF9ABD2, "a2":0xF9C5B6,  "value":'''WaitForSingleObjectEx''' }]

blacklisted = [0xF9AB9F,0xF9ABD2]


def set_hexrays_comment(address, text):
    '''
    set comment in decompiled code
    '''
    cfunc = idaapi.decompile(address)
    tl = idaapi.treeloc_t()
    tl.ea = address
    tl.itp = idaapi.ITP_SEMI
    cfunc.set_user_cmt(tl, text)
    cfunc.save_user_cmts() 


def set_comment(address, text):
    ## Set in dissassembly
    idc.set_cmt(address, text,0)
    ## Set in decompiled data
    set_hexrays_comment(address, text)


for s in strings:
    addr = s.get('a1') 
    if addr in blacklisted:
        addr = s.get('a2')
        set_comment(addr, s.get('value'))



## APIs

Some APIs are resolved using hashes with a simple hash algorithm.

0x4A6BED0C - `GetProcAddress`

0xE1A00B78 - `LoadLibraryA`

0x4D6CD9E - `HeapFree`

The rest are resolved using `GetProcAddress` with decrypted strings (API names).

In [1]:
def get_hash(string):
    out = 0xb6
    string = string.lower()
    for c in string:
        out = (ord(c) + out * 5) & 0xffffffff
    return out

hex(get_hash('HeapFree'))

'0x4d6cd9e'